In [3]:
import cv2
import numpy as np
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders
from ultralytics import YOLO
import time

# Load YOLOv8 model
model = YOLO('yolov8n.pt')

# Email configuration
def send_email_alert(image_path):
    sender_email = 'behappy20041@gmail.com'
    receiver_email = 'rupeshsinghdk4@gmail.com'
    password = 'syaa ryxd eqpk zixz'
    
    subject = "Alert: Restricted Area Breach Detected"
    body = "Someone has crossed the restricted area."

    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = subject

    msg.attach(MIMEText(body, 'plain'))

    # Attach the screenshot
    attachment = open(image_path, "rb")
    part = MIMEBase('application', 'octet-stream')
    part.set_payload((attachment).read())
    encoders.encode_base64(part)
    part.add_header('Content-Disposition', f"attachment; filename= {image_path}")
    msg.attach(part)

    try:
        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()
        server.login(sender_email, password)
        text = msg.as_string()
        server.sendmail(sender_email, receiver_email, text)
        server.quit()
        print("Email with screenshot sent successfully.")
    except Exception as e:
        print(f"Failed to send email: {e}")

# Define the restricted area rectangle
RECTANGLE_START = (100, 200)  # Top-left corner of the rectangle (x, y)
RECTANGLE_END = (500, 400)    # Bottom-right corner of the rectangle (x, y)

# Initialize the video capture
cap = cv2.VideoCapture(0)

# Timing setup
last_sent_time = 0
interval = 5  # 5 seconds interval

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Perform detection
    results = model(frame)

    # Draw the restricted area rectangle
    cv2.rectangle(frame, RECTANGLE_START, RECTANGLE_END, (0, 0, 255), 2)

    for result in results:
        boxes = result.boxes
        for box in boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])

            # Check if the detected object is a person
            if int(box.cls[0]) == 0:  # Class 0 is 'person' in COCO dataset
                # Draw bounding box
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

                # Check if the person has crossed the rectangle
                if (x2 > RECTANGLE_START[0] and x1 < RECTANGLE_END[0] and
                    y2 > RECTANGLE_START[1] and y1 < RECTANGLE_END[1]):
                    current_time = time.time()
                    if current_time - last_sent_time > interval:
                        # Save the frame as an image
                        image_path = "breach.png"
                        cv2.imwrite(image_path, frame)
                        send_email_alert(image_path)
                        last_sent_time = current_time

    cv2.imshow("Restricted Area Surveillance", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


0: 480x640 1 airplane, 438.1ms
Speed: 17.9ms preprocess, 438.1ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 airplane, 376.0ms
Speed: 8.0ms preprocess, 376.0ms inference, 1.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 airplane, 270.3ms
Speed: 4.0ms preprocess, 270.3ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 airplane, 272.9ms
Speed: 4.0ms preprocess, 272.9ms inference, 1.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 airplane, 274.4ms
Speed: 6.0ms preprocess, 274.4ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 airplane, 265.4ms
Speed: 4.0ms preprocess, 265.4ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 airplane, 272.5ms
Speed: 5.6ms preprocess, 272.5ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 airplane, 285.4ms
Speed: 6.3ms preprocess, 285.4ms inference, 2.7ms postpro